In [1]:
import numpy as np
import matplotlib.pyplot as plt
import copy
import sklearn.metrics.pairwise 
import scipy
%matplotlib inline
plt.style.use(['ggplot'])

In [2]:
N = 10
X = np.random.randint(1, 51, (N, 2)) # (2 X N)
print(X)

[[16 14]
 [48 30]
 [41 30]
 [37 30]
 [37 23]
 [ 8 34]
 [20 14]
 [46 31]
 [14  9]
 [44  2]]


In [3]:
Reflections = {
    'x': np.array([[1.0, 0.0], 
                       [0.0, -1.0]]),  # reflection on x ais
    
    'y': np.array([[-1.0, 0.0], 
                        [0.0, 1.0]]),  # reflection on y axis
    
    'y=x': np.array([[0.0, 1.0], 
                        [1.0, 0.0]]),  # reflection on y=x
    
    'y=-x': np.array([[0.0, -1.0], 
                        [-1.0, 0.0]]),  # reflection on y=-x
    
    'origin': np.array([[-1.0, 0.0], 
                         [0.0, -1.0]]), # reflection on origin 

}

Tranlations = {
    'x': np.array([[-1.0, 0.0], 
                        [0.0, 0.0]]),  
}

def  cal_cost(theta,X,y):
    '''
    
    Calculates the cost for given X and Y. The following shows and example of a single dimensional X
    theta = Vector of thetas 
    X     = Row of X's np.zeros((2,j))
    y     = Actual y's np.zeros((2,1))
    
    where:
        j is the no of features
    '''
    
    m = len(y)
    
    predictions = X.dot(theta)
    cost = (1/2*m) * np.sum(np.square(predictions-y))
    return cost

def gradient_descent(X,y,theta,learning_rate=0.01,iterations=100):
    '''
    X    = Matrix of X with added bias units
    y    = Vector of Y
    theta=Vector of thetas np.random.randn(j,1)
    learning_rate 
    iterations = no of iterations
    
    Returns the final theta vector and array of cost history over no of iterations
    '''
    m = len(y)
    cost_history = np.zeros(iterations)
    theta_history = np.zeros((iterations,2))
    for it in range(iterations):
        
        prediction = np.dot(X,theta)
        
        #theta = theta -(1/m)*learning_rate*( X.T.dot((prediction - y)))
        theta = theta -(1/m)*learning_rate*( X.T.dot((prediction - y)))
        #theta_history[it,:] =theta.T
        cost_history[it]  = cal_cost(theta,X,y)
        
    return theta, cost_history#, theta_history



In [ ]:
lr = 0.001
n_iter = 100
tolerance  = 0.001

# source policy
Ks = np.eye(2)
#Ks = np.array([[0.001, -3.0], [0.0, 0.348]])
us = X@Ks.T 

# translated target policy
K_rot = R[op[0]] + Ks  
u_rot = X@K_rot.T 

# reflected target policy
K_ref = R[op[1]] @ Ks  
u_ref = X@K_ref.T 

# print("src u: ", us[0])
print("trans: \n", K_rot,"\n\n u:", u_rot[0])
print("\nref: \n", K_ref, "\n\n u:", u_ref[0])

theta_rot, cost_history_rot = gradient_descent(X, u_rot, Ks, lr, n_iter)
theta_ref, cost_history_ref = gradient_descent(X, u_ref, Ks, lr, n_iter)
print("theta trans: ", theta_rot)
rot_steps = np.where(cost_history_rot <= tolerance)[0][0]
ref_steps = np.where(cost_history_ref <= tolerance)[0][0]

n_gradient_steps = [rot_steps, ref_steps]
n_gradient_steps